In [17]:
import bs4
import requests
import re
import tqdm

In [18]:
def get_string(c):
    return ''.join(i.string if i.string is not None else "" for i in c.children)

def convert(section):
    children = list(section.children)
    if len(children) < 2:
        return

    # print(section.attrs)
    # title = children[1].string
    # if title is None or len(title.split()) > 1:
    #     return

    title = section.attrs['id'].split('/')[-1]
    data = False
    if title.startswith('body.'):
        title = title[5:]
    if title == 'request_body':
        title = 'ComputeRoutesRequest' # todo
    if 'Response' in title:
        data = True

    # print(title)

    code_type = ""
    enum_vals = {}
    fields = {}

    for i in children[2:]:
        if i.name != 'section':
            continue

        if 'description' in i.attrs['id']:
            p = list(i.children)[1]
            description = get_string(p)
        elif 'ENUM_VALUES' in i.attrs['id']:
            code_type = "enum"
            for row in i.find("tbody").find_all("tr"):
                cs = list(row.children)
                td1, td2 = cs[1], cs[3]
                enum_vals[get_string(td1)] = get_string(td2)
        elif 'FIELDS' in i.attrs['id']:
            code_type = "struct"
            for row in i.find('tbody').find_all('tr'):
                cs = list(row.children)
                try:
                    td1, td2 = cs[1], cs[3]
                    cs2 = list(td2.children)
                    cs3 = list(cs2[1].children)
                    fields[get_string(td1)] = (get_string(cs3[0]), get_string(cs2[3]))
                except:
                    # title += "\n" + get_string(cs[1])
                    pass # todo comment?

    return (title, code_type, enum_vals, fields, data)

def get_things(sections):
    things = {}

    for child in sections:
        if child.name == 'section' and hasattr(child, 'attrs') and 'id' in child.attrs:
            if vals := convert(child):
                t, ct, ev, f, d = vals
                things[t] = (ct, ev, f, d)
    return things

In [19]:
def convert_type(name, type):
    if type == 'object' or re.match(r'object\ \(.*\ .*\)', type):
        t = '[String: Any]'
    elif type.startswith('object') or type.startswith('enum'):
        t = type.split('(')[1][:-1]
    elif type.startswith('string'):
        t = 'String'
    elif type == 'integer' or type == 'number':
        t = 'Int'
    elif type == 'boolean':
        t = 'Bool'
    else:
        print("unknown type", type)
        t = '[String: Any]'
    n = name
    if name.endswith("[]"):
        n = n[:-2]
        t = '[' + t + ']'
    return n, t
        
def get_file(things):
    ss = ""
    for k, v in things.items():
        ct, ev, f, d = v
        s = ""
        if ct == "enum":
            s += f"enum {k}: String, Codable {{\n"
            for name, comment in ev.items():
                s += f"\tcase {name} // {comment}\n"
            s += "}"
        elif ct == "struct":
            s += f"struct {k}: Codable {{\n"
            manual = False
            for name, (type, comment) in f.items():
                n, c = convert_type(name, type)
                if '[String: Any]' in c:
                    manual = True
                s += f"\tvar {n}: {c}? = nil // {comment}\n"
            if manual:
                s += "\n\tenum CodingKeys: String, CodingKey {\n"
                for name in f.keys():
                    s += f"\t\tcase {name.strip("[]")}\n"
                s += "\t}\n\n"
                s += "\tinit(from decoder: Decoder) throws {\n\t\tlet container = try decoder.container(keyedBy: CodingKeys.self)\n\n"
                for name, (type, comment) in f.items():
                    n, c = convert_type(name, type)
                    s += f"\t\t{n} = try container.decodeIfPresent({c}.self, forKey: .{n})\n"
                s += "}\n\n\tfunc encode(to encoder: Encoder) throws {\n\t\tvar container = encoder.container(keyedBy: CodingKeys.self)\n\n"
                for name in f.keys():
                    n = name.strip("[]")
                    s += f"\t\ttry container.encodeIfPresent({n}, forKey: .{n})\n"
                s += "\t}\n"
            s += "}\n"
            if d:
                s += f"""
extension {k} {{
    static func from(jsonData data: Data) -> ComputeRoutesResponse? {{
        return try? JSONDecoder().decode({k}.self, from: data)
    }}
}}


"""
        if s != "":
            ss += s + "\n\n"
    return ss

In [20]:
def find_section(soup):
    return [i for i in soup.find("div", attrs={'class': 'devsite-article-body'}).children if i.name == 'section'][0]

def to_swift(url, full):
    soup = bs4.BeautifulSoup(requests.get(url).text)
    sections = soup.find_all("section", recursive=True)
    things = get_things(sections)
    return get_file(things)

In [21]:
urls = [
    "https://developers.google.com/maps/documentation/routes/reference/rest/v2/FallbackInfo",
    "https://developers.google.com/maps/documentation/routes/reference/rest/v2/LatLng",
    "https://developers.google.com/maps/documentation/routes/reference/rest/v2/LocalizedText",
    "https://developers.google.com/maps/documentation/routes/reference/rest/v2/Location",
    "https://developers.google.com/maps/documentation/routes/reference/rest/v2/Money",
    "https://developers.google.com/maps/documentation/routes/reference/rest/v2/RouteModifiers",
    "https://developers.google.com/maps/documentation/routes/reference/rest/v2/RouteTravelAdvisory",
    "https://developers.google.com/maps/documentation/routes/reference/rest/v2/RouteTravelMode",
    "https://developers.google.com/maps/documentation/routes/reference/rest/v2/RoutingPreference",
    "https://developers.google.com/maps/documentation/routes/reference/rest/v2/SpeedReadingInterval",
    "https://developers.google.com/maps/documentation/routes/reference/rest/v2/Status",
    "https://developers.google.com/maps/documentation/routes/reference/rest/v2/TollInfo",
    "https://developers.google.com/maps/documentation/routes/reference/rest/v2/TrafficModel",
    "https://developers.google.com/maps/documentation/routes/reference/rest/v2/TransitPreferences",
    "https://developers.google.com/maps/documentation/routes/reference/rest/v2/Waypoint"
]

sss = "import Foundation\n"
sss += to_swift("https://developers.google.com/maps/documentation/routes/reference/rest/v2/TopLevel/computeRoutes", True)
for u in tqdm.tqdm(urls):
    sss += to_swift(u, False) + "\n"

open("test", "w").write(sss)

100%|███████████████████████████████████████████| 15/15 [00:08<00:00,  1.87it/s]


44285